# TSFEDL Models

In [1]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from libraries.utils import *





## Load data

In [2]:
# ############ configuration - trace ################
# ############################################


CODE = 'theft_protection'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 3                     ### format of data collection

base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True: varibale window size, False: fixed window size; wether to use variable window size or not

#####################################################


ref_samples_basepath = os.path.join(normalbase_path, 'diag_refsamples')
ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, 'diag_subseq')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')


print('ref_samples_path:\n', ref_samples_basepath)
print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   

train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]

# ### remove.Ds_store from all lists
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]


varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

if IS_VAR_WINDOW:
    train_data_path = ref_var_samples_path
else:
    train_data_path = ref_samples_path

test_data_path = test_subseq_path

print('train_data:\n', train_data_path)
print(len(train_data_path))
print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)



../../trace_data/theft_protection/single_thread/version_3/normal
../../trace_data/theft_protection/single_thread/version_3/faulty_data
ref_samples_path:
 ../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples
ref_var_samples_path:
 ../../trace_data/theft_protection/single_thread/version_3/normal/diag_var_refsamples
diag_subseq_path:
 ../../trace_data/theft_protection/single_thread/version_3/faulty_data/diag_subseq
train_data:
 ['../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/379.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/396.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/115.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/400.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsamples/142.json', '../../trace_data/theft_protection/single_thread/version_3/normal/diag_refsam

In [3]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3 or VER == 4:
    check_con, _ = is_consistent([train_varlist_path[0]]+ varlist_path) ### compare with train varlist

    if check_con != False:
        to_number = read_json(varlist_path[0])
        from_number = mapint2var(to_number)
    else:
        ### load normal varlist
        print('loading normal varlist')
        to_number = read_json(train_varlist_path[0])
        from_number = mapint2var(to_number)



varlist 1 is consistent with varlist 0
varlist 2 is consistent with varlist 0
varlist 3 is consistent with varlist 0


In [4]:
to_number = read_json(train_varlist_path[0])
from_number = mapint2var(to_number)

In [5]:
# #### key finder ####
# from_number[44]

In [6]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Prepare Training Data

In [ ]:
'''
TODO:
1. check the code for feature extraction in Approach 1
2. split the ref_samples in seq of 50 events (sliding interval of 1)
3. prepare y_train i.e. the expected output of the seq
4. train the model
'''

In [7]:
### load all the reference samples (fixed window size)
ref_samples = []
for ref_sample in train_data_path:
    ref_samples.append(read_traces(ref_sample))

In [10]:
np.array(ref_samples).shape

(438, 2, 500)

## Train DL Models

In [1]:
import tensorflow as tf
import TSFEDL.models_keras as tsfedl

In [7]:
#### build model ####
input = tf.keras.Input(shape=(50, 2))
model = tsfedl.HuangMeiLing(input_tensor=input, include_top=False)
x = model.output
x = tf.keras.layers.LSTM(units=20)(x)
### Add the top module
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(50)(x)
x = tf.keras.layers.Dense(10)(x)
x = tf.keras.layers.Dense(2)(x)
out = tf.keras.layers.Reshape([2, 1])(x)

### create new model
new_model = tf.keras.Model(inputs=input, outputs=out, name="forecaster")

print(model.summary())
print(new_model.summary())


Model: "HuangMeiLing"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 50, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d_10               │ (None, 52, 2)          │             0 │
│ (ZeroPadding1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 7, 48)          │         1,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 6, 48)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d_11               │ (None, 12, 48)         │             0 │
│ (ZeroPadding1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 3, 256)         │        86,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 2, 256)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 87,760 (342.81 KB)

 Trainable params: 87,760 (342.81 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 50, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d_10               │ (None, 52, 2)          │             0 │
│ (ZeroPadding1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 7, 48)          │         1,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 6, 48)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ zero_padding1d_11               │ (None, 12, 48)         │             0 │
│ (ZeroPadding1D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 3, 256)         │        86,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 2, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 20)             │        22,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 50)             │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 10)             │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 2)              │            22 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_5 (Reshape)             │ (None, 2, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,502 (435.55 KB)

 Trainable params: 111,502 (435.55 KB)

 Non-trainable params: 0 (0.00 B)

None
